In [ ]:
# Imported necessary libaries and set settings for each seaborn visual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.5)

# Imported all the data 
bom_df = pd.read_csv("zipped_df/bom.movie_gross.csv")
imdb_name_basics = pd.read_csv('zipped_df/imdb.name.basics.csv')
imdb_akas = pd.read_csv('zipped_df/imdb.title.akas.csv')
imdb_title_basics = pd.read_csv('zipped_df/imdb.title.basics.csv')
imdb_title_crew = pd.read_csv('zipped_df/imdb.title.crew.csv')
imdb_title_principal = pd.read_csv('zipped_df/imdb.title.principals.csv')
imdb_title_ratings = pd.read_csv('zipped_df/imdb.title.ratings.csv')
rt_movie_info = pd.read_csv('zipped_df/rt.movie_info.tsv', sep='\t')
rt_reviews = pd.read_csv("zipped_df/rt.reviews.tsv", sep="\t", encoding = "ISO-8859-1")
tmdb_movies = pd.read_csv('zipped_df/tmdb.movies.csv')
tn_movie_budgets = pd.read_csv('zipped_df/tn.movie_budgets.csv')

# Changed the column 'release_date' to DateTime format
tn_movie_budgets['release_date'] = pd.to_datetime(tn_movie_budgets['release_date'])

# Cleaned the years to only have years 2010-2018
tn_movie_budgets = tn_movie_budgets[(tn_movie_budgets['release_date'].dt.year > 2009) & (tn_movie_budgets['release_date'].dt.year < 2019)]

# Changed data types to int
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].str.replace(',', '').str.replace('$', '').astype(int)
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].str.replace(',', '').str.replace('$', '').astype(int)
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(int)

# Created a new column to show the ratio betweem worldwide_gross and movie budget
tn_movie_budgets['ww_gross/budget'] = (tn_movie_budgets['worldwide_gross']/tn_movie_budgets['production_budget'])

# Added new year column in order to facilitate a join between different data sets
tn_movie_budgets['year'] = tn_movie_budgets.release_date.dt.year

# Created a new column to display how much each moive made
tn_movie_budgets['Earnings'] = (tn_movie_budgets['worldwide_gross'] - tn_movie_budgets['production_budget'])

# Merged two tables into a single data frame to compare data
imdb_title_ratings_basics = pd.merge(imdb_title_ratings, imdb_title_basics, left_on='tconst', right_on='tconst')

# Merged another data frame to create a master data frame
master_df = pd.merge(imdb_title_ratings_basics, tn_movie_budgets, left_on=['primary_title', 'start_year'], right_on=['movie', 'year'])

# Renamed the colums to better navigate the data frame when manipuating 
master_df.rename(columns={'averagerating':'Average Rating'}, inplace=True)
master_df.rename(columns={'Numvotes':'Number of Votes'}, inplace=True)
master_df.rename(str.title, axis='columns', inplace=True)

# Dropped unnecessary columns
master_df.drop(columns=['Primary_Title', 'Original_Title', 'Start_Year', 'Id', 'Release_Date', 'Tconst'], inplace=True)

# Created three columns to accuratly count the genre for each movie, then dropped the original row to not have duplicates
master_df['Genres'].str.split(',', expand=True)
master_df["Genre_1"] = master_df.Genres.str.split(',', expand=True)[0]
master_df["Genre_2"] = master_df.Genres.str.split(',', expand=True)[1]
master_df["Genre_3"] = master_df.Genres.str.split(',', expand=True)[2]
master_df = master_df.drop(columns = ["Genres"])

# Found the mean, median, std, max and min for each of the following columns
master_df['Average Rating'].apply(['mean', 'median', 'std', 'max', 'min'])
master_df['Number of Votes'].apply(['mean', 'median', 'std', 'max', 'min'])
master_df['Runtime_Minutes'].apply(['mean', 'median', 'std', 'max', 'min'])
master_df['Production_Budget'].apply(['mean', 'median', 'std', 'max', 'min'])

# Created a new genre set to calculate the number of genres for each movie (lines 71-104)
genre_set = set(master_df["Genre_1"].unique())
genre_set.update(master_df["Genre_2"].unique())

genre_set.update(master_df["Genre_3"].unique())
genre_dict = { i : 0 for i in genre_set }
genre_number = {i : 0 for i in genre_set }
earnings_dict = {i: 0 for i in genre_set}

temp = master_df[["Genre_1", "Genre_2", "Genre_3", "Average Rating", "Earnings"]]
for i,j in temp.iterrows():
    genre_number[j[0]]+=1
    genre_number[j[1]]+=1
    genre_number[j[2]]+=1
    genre_dict[j[0]]+=j[3]
    genre_dict[j[1]]+=j[3]
    genre_dict[j[2]]+=j[3]
    earnings_dict[j[0]]+=j[4]
    earnings_dict[j[1]]+=j[4]
    earnings_dict[j[2]]+=j[4]

for key in genre_dict.keys():
    genre_dict[key] = genre_dict[key]/genre_number[key]
    earnings_dict[key] = earnings_dict[key]/genre_number[key]

genre_dict = {k: genre_dict[k] for k in genre_dict if not pd.isna(k)}
earnings_dict = {k: earnings_dict[k] for k in earnings_dict if not pd.isna(k)}

for key, value in earnings_dict.items():
     print(key, value)

avg_earnings_by_genres_df = pd.DataFrame.from_dict(earnings_dict, orient = 'index')

avg_earnings_by_genres_df.rename(columns = {avg_earnings_by_genres_df.columns[0]:"Earnings"}, inplace=True)
avg_earnings_by_genres_df['Category'] = avg_earnings_by_genres_df.index

# Plotted the genres by earnings
x4 = avg_earnings_by_genres_df['Category']
y4 = avg_earnings_by_genres_df['Earnings']

# Plotted the figure to show the average earnings for each genre, then saved the figure
plt.figure(figsize=(15,10))
ax4 = sns.barplot(x4, y4, data=master_df)
plt.title('Earnings by Genre')
plt.xlabel('Genre')
plt.xticks(rotation=45)
plt.ylabel('Earnings')
plt.show()

fig = ax4.get_figure()
fig.savefig("Earnings_by_Genre.png")
# This plots the Earnings by Genre visual

# ADDED NEW DATA FRAME 
# We wanted to explore how actors and directs impacted earnings and ratings
all_movies_df = pd.read_csv('zipped_df/all_movie.csv.zip')

# Cleaned the data to only have years 2010 - 2018
rt_df = all_movies_df[(all_movies_df['Year'] > 2009) & (all_movies_df['Year']< 2019)]

# Merged the new data frame to the original master to create a master2 data frame
master2_df = pd.merge(master_df, rt_df, left_on=("Movie", "Year"), right_on=("Title","Year"))

#Taking out the duplicate rows, based on the Title and Year
master2_df = master2_df[~master2_df[['Title', 'Year']].duplicated()]

# Dropped columns that we did not need
master2_df.drop(columns=['Director 2', 'Director 3', 'Description', 'Writer 2', 'Writer 3', 'Writer 4', 'Rating', 'Genre', 'Release Date', 'Runtime', 'Title', 'Cast 3', 'Cast 4', 'Cast 5', 'Cast 6'], inplace=True)

# Found average earnings for the 20 most common actors in movies in our database
common_leads = master2_df['Cast 1'].apply(['value_counts'])[0:20].reset_index()

# Sorted the earnings by specific actor
actor_earnings = master2_df[['Cast 1', 'Earnings']].sort_values(by='Earnings', ascending=False)

# Merged both of the data frames into one to be able to pull specifc data
new_df = pd.merge(common_leads, actor_earnings, left_on='index', right_on='Cast 1')

# Called a new dataframe to only have relevant data
condesnsed_new = new_df[['Cast 1', 'Earnings']]

# Created a new variable in order to plot the cast vs the average earnings
new_var = condesnsed_new.groupby('Cast 1').sum().sort_values(by='Cast 1', ascending=False)
new_var = new_var.reindex(common_leads["index"])

# Plotted the figure
plt.figure(figsize=(18,12))
ax7 = sns.barplot(x=new_var.index, y='Earnings', data=new_var, hue=common_leads['value_counts'], palette='colorblind')
plt.legend(title='Number of Movies')
plt.title('Career Earnings')
plt.xticks(rotation = 90)
plt.xlabel('Actor/Acress')
plt.show()
# This plots the average earnings per actor

# Found average earnings for the 20 most common directors in movies in our database
common_directors = master2_df['Director 1'].apply(['value_counts'])[:20].reset_index()

director_earnings = master2_df[['Director 1', 'Earnings']].sort_values(by='Earnings', ascending=False)

new2_df = pd.merge(common_directors, director_earnings, left_on='index', right_on='Director 1')

condesnsed_new2 = new2_df[['Director 1', 'Earnings']]

new2_var = condesnsed_new2.groupby('Director 1').sum().sort_values(by='Director 1', ascending=False)
new2_var = new2_var.reindex(common_directors["index"])

plt.figure(figsize=(18,12))
ax8 = sns.barplot(x=new2_var.index, y='Earnings', data=new2_var, hue=common_directors['value_counts'], palette='colorblind')
plt.legend(title='Number of Movies')
plt.title('Career Earnings')
plt.xticks(rotation = 90)
plt.xlabel('Director')
plt.show()
# This plots the average earnings per director

# Found average ratings for the 20 most common actors in movies in our database
common_leads_ratings = master2_df['Cast 1'].apply(['value_counts'])[0:20].reset_index()

actor_ratings = master2_df[['Cast 1', 'Average Rating']].sort_values(by='Average Rating', ascending=False)

new3_df = pd.merge(common_leads_ratings, actor_ratings, left_on='index', right_on='Cast 1', how='inner')

condesnsed3_new = new3_df[['Cast 1', 'Average Rating']]

new3_var = condesnsed3_new.groupby('Cast 1').mean().sort_values(by='Cast 1', ascending=False)
new3_var = new3_var.reindex(common_leads["index"])

plt.figure(figsize=(18,12))
ax9 = sns.barplot(x=new3_var.index, y='Average Rating', data=new3_var, hue=common_leads_ratings['value_counts'], palette='colorblind')
ax9.set(ylim=(0, 10))
plt.legend(title='Number of Movies')
plt.title('Career Rating')
plt.xticks(rotation = 90)
plt.xlabel('Actor/Acress')
plt.show()
# This plots the average rating per actor

# Found average ratings for the 20 most common directors in movies in our database
common_director_ratings = master2_df['Director 1'].apply(['value_counts'])[0:20].reset_index()

director_ratings = master2_df[['Director 1', 'Average Rating']].sort_values(by='Average Rating', ascending=False)

new4_df = pd.merge(common_director_ratings, director_ratings, left_on='index', right_on='Director 1')

condesnsed4_new = new4_df[['Director 1', 'Average Rating']]

new4_var = condesnsed4_new.groupby('Director 1').mean().sort_values(by='Director 1', ascending=False)
new4_var = new4_var.reindex(common_director_ratings["index"])

plt.figure(figsize=(18,12))
ax10 = sns.barplot(x=new4_var.index, y='Average Rating', data=new4_var, hue=common_leads_ratings['value_counts'], palette='colorblind')
plt.legend(title='Number of Movies')
plt.title('Career Rating')
plt.xlabel('Director')
plt.xticks(rotation = 90)
plt.show()
# This plots the average rating per director